In [22]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_log_error, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.ensemble import AdaBoostRegressor 
from catboost import CatBoost, CatBoostRegressor 
from xgboost import XGBRegressor

import warnings 
import holidays 
warnings.filterwarnings ("ignore")

In [23]:
import os
os.chdir('C:/Users/cui62/Desktop/zentativity/Kaggle_Forecasting Mini-Course Sales')

In [51]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

In [52]:
def feature_engineering(df) :
    df.date = pd.to_datetime (df.date)
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month
    df['dayofweek'] = df.date.dt.dayofweek
    df['dayofmonth'] = df.date.dt.day
    df['dayofyear'] = df.date.dt.dayofyear
    df['month_sin'] = np.sin(2*np.pi*df.month/12)
    df['month_cos'] = np.cos(2*np.pi*df.month/12)
    df['day_sin'] = np.sin(2*np.pi*df.dayofmonth/24)
    df['day_cos'] = np.cos (2*np.pi+df.dayofmonth/24)
    return df


def get_holidays(df):
    # Country holiday based features
    years_list = [2017, 2018, 2019, 2020, 2021, 2022]

    # Define holiday instances for each country
    holiday_AR = holidays.CountryHoliday('AR', years=years_list)
    holiday_CA = holidays.CountryHoliday('CA', years=years_list)
    holiday_EE = holidays.CountryHoliday('EE', years=years_list)
    holiday_JP = holidays.CountryHoliday('JP', years=years_list)
    holiday_ES = holidays.CountryHoliday('ES', years=years_list)

    # Merge all holiday dictionaries into a single one
    holiday_dict = holiday_AR.copy()
    holiday_dict.update(holiday_CA)
    holiday_dict.update(holiday_EE)
    holiday_dict.update(holiday_JP)
    holiday_dict.update(holiday_ES)

    df['holiday_name'] = df['date'].map (holiday_dict)
    df['is _holiday'] = np.where(df['holiday_name'].notnull(),1,0)
    df['holiday_name'] = df['holiday_name'].fillna('Not Holiday')
    df=df.drop(columns=['holiday_name'])
    
    return df

In [53]:
df_gdp = pd.DataFrame({'country': np.repeat(['Argentina', 'Canada', 'Estonia', 'Japan', 'Spain'], 6).tolist(),
                           'year': [2017, 2018, 2019, 2020, 2021, 2022] * 5, 
                           'gdp': [643.63, 524.82, 447.75, 385.54, 487.23, 632.77,
                                   1.65*np.power(10, 3), 1.73*np.power(10, 3), 1.74*np.power(10, 3), 1.65*np.power(10, 3), 2*np.power(10, 3), 2.14*np.power(10, 3),
                                   26.92, 30.62, 31.08, 31.37, 37.19, 38.1,
                                   4.93*np.power(10, 3), 5.04*np.power(10, 3), 5.12*np.power(10, 3), 5.05*np.power(10, 3), 5.01*np.power(10, 3), 4.23*np.power(10, 3),
                                   1.31*np.power(10, 3), 1.42*np.power(10, 3), 1.39*np.power(10, 3), 1.28*np.power(10, 3), 1.43*np.power(10, 3), 1.4*np.power(10, 3)]
                           })
df_gdp['gdp'] = np.log(df_gdp['gdp'])
df_gdp.head()

,country,year,gdp
0,Argentina,2017,6.467124
1,Argentina,2018,6.263055
2,Argentina,2019,6.104235
3,Argentina,2020,5.954645
4,Argentina,2021,6.188736


In [54]:
df_train = feature_engineering(train)
df_test = feature_engineering(test)

In [55]:
df_train = get_holidays(df_train)
df_test = get_holidays(df_test)

In [56]:
df_train = pd.merge(df_train, df_gdp, on=['country','year'], how='left')
df_test = pd.merge(df_test, df_gdp, on=['country','year'], how='left')


In [57]:
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

In [58]:
#Deleting Covid
df_train = df_train[~df_train['date'].between('2020-03-01','2020-05-31')]


print(f'[INFO] Shapes:'
      f'\n train: {df_train.shape}'
      f'\n test: {df_test.shape}\n')

print(f'[INFO] Any missing values:'
      f'\n train: {df_train.isna().any().any()}'
      f'\n test: {df_test.isna().any().any()}\n')

[INFO] Shapes:
 train: (130050, 27)
 test: (27375, 26)

[INFO] Any missing values:
 train: False
 test: False



In [69]:
df_train.head()

,id,date,num_sold,year,month,dayofweek,dayofmonth,dayofyear,month_sin,month_cos,...,country_Japan,country_Spain,store_Kagglazon,store_Kaggle Learn,store_Kaggle Store,product_Using LLMs to Improve Your Coding,product_Using LLMs to Train More LLMs,product_Using LLMs to Win Friends and Influence People,product_Using LLMs to Win More Kaggle Competitions,product_Using LLMs to Write Better
0,0,2017-01-01,63,2017,1,6,1,1,0.5,0.866025,...,0,0,0,1,0,1,0,0,0,0
1,1,2017-01-01,66,2017,1,6,1,1,0.5,0.866025,...,0,0,0,1,0,0,1,0,0,0
2,2,2017-01-01,9,2017,1,6,1,1,0.5,0.866025,...,0,0,0,1,0,0,0,1,0,0
3,3,2017-01-01,59,2017,1,6,1,1,0.5,0.866025,...,0,0,0,1,0,0,0,0,1,0
4,4,2017-01-01,49,2017,1,6,1,1,0.5,0.866025,...,0,0,0,1,0,0,0,0,0,1


In [19]:
df_test.head()

,id,date,year,month,dayofweek,dayofmonth,dayofyear,month_sin,month_cos,day_sin,...,country_Japan,country_Spain,store_Kagglazon,store_Kaggle Learn,store_Kaggle Store,product_Using LLMs to Improve Your Coding,product_Using LLMs to Train More LLMs,product_Using LLMs to Win Friends and Influence People,product_Using LLMs to Win More Kaggle Competitions,product_Using LLMs to Write Better
0,136950,2022-01-01,2022,1,5,1,1,0.5,0.866025,0.258819,...,0,0,0,1,0,1,0,0,0,0
1,136951,2022-01-01,2022,1,5,1,1,0.5,0.866025,0.258819,...,0,0,0,1,0,0,1,0,0,0
2,136952,2022-01-01,2022,1,5,1,1,0.5,0.866025,0.258819,...,0,0,0,1,0,0,0,1,0,0
3,136953,2022-01-01,2022,1,5,1,1,0.5,0.866025,0.258819,...,0,0,0,1,0,0,0,0,1,0
4,136954,2022-01-01,2022,1,5,1,1,0.5,0.866025,0.258819,...,0,0,0,1,0,0,0,0,0,1


In [59]:
X_train = df_train.drop(['date','num_sold'], axis=1)

In [60]:
df_test = df_test.drop(['date'], axis=1)

In [61]:
y_train = df_train['num_sold'].apply(lambda x: np.log1p(x)).values

In [62]:
X_train['gdp'] = df_train['gdp'].apply(lambda x: np.log1p (x)).values
df_test['gdp'] = df_test['gdp'].apply(lambda x: np. log1p(x)).values

In [44]:
#Define SMAPE
def smape(y_true, y_pred):
    denominator = (np.abs(y_true)+ np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred)/denominator
    diff[denominator == 0] = 0.0
    return np.nanmean(diff)

# Make the RMSLE scorer
rmsle_scorer = make_scorer (smape)


# Initialize and evaluate different regression models using cross-valida
models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(),
    'XGBRegressor': XGBRegressor(),
    'CatBoostRegressor': CatBoostRegressor ()
}

for model_name, model in models.items():
    # Perform cross-validation with SMAPE scorer
    cv_scores = cross_val_score(model, X_train, y_train, scoring=rmsle_scorer)
    
    print(f"Model: {model_name}")
    print(f"Cross-Validation SMAPE: {cv_scores}")
    print(f"Average SMAPE: {np.mean(cv_scores)}\n")
    print("-------------------------")

Model: Linear Regression
Cross-Validation SMAPE: [2.56189949 2.50664829 2.24951047 2.33038658 2.45470944]
Average SMAPE: 2.4206308537576087

-------------------------
Model: Decision Tree
Cross-Validation SMAPE: [2.81569256 2.37360183 2.22284983 2.60432444 3.12191235]
Average SMAPE: 2.627676201368867

-------------------------
Model: Random Forest
Cross-Validation SMAPE: [2.42485529 1.94158667 1.81792893 2.38060106 2.37181748]
Average SMAPE: 2.187357886386887

-------------------------
Model: XGBRegressor
Cross-Validation SMAPE: [2.32310827 1.75790344 1.4960915  2.25418701 1.57472665]
Average SMAPE: 1.8812033721338408

-------------------------
Learning rate set to 0.08529
0:	learn: 1.0511116	total: 173ms	remaining: 2m 52s
1:	learn: 0.9653708	total: 188ms	remaining: 1m 33s
2:	learn: 0.8867814	total: 201ms	remaining: 1m 6s
3:	learn: 0.8150381	total: 213ms	remaining: 53.1s
4:	learn: 0.7501461	total: 226ms	remaining: 45s
5:	learn: 0.6902242	total: 239ms	remaining: 39.5s
6:	learn: 0.635752

152:	learn: 0.0609356	total: 9.47s	remaining: 52.4s
153:	learn: 0.0608640	total: 9.49s	remaining: 52.1s
154:	learn: 0.0608330	total: 9.51s	remaining: 51.8s
155:	learn: 0.0607350	total: 9.52s	remaining: 51.5s
156:	learn: 0.0606485	total: 9.54s	remaining: 51.2s
157:	learn: 0.0605923	total: 9.56s	remaining: 50.9s
158:	learn: 0.0605275	total: 9.57s	remaining: 50.6s
159:	learn: 0.0604552	total: 9.6s	remaining: 50.4s
160:	learn: 0.0604034	total: 9.61s	remaining: 50.1s
161:	learn: 0.0602907	total: 9.63s	remaining: 49.8s
162:	learn: 0.0602312	total: 9.66s	remaining: 49.6s
163:	learn: 0.0601602	total: 9.67s	remaining: 49.3s
164:	learn: 0.0600961	total: 9.69s	remaining: 49.1s
165:	learn: 0.0600004	total: 9.71s	remaining: 48.8s
166:	learn: 0.0598992	total: 9.73s	remaining: 48.5s
167:	learn: 0.0598581	total: 9.74s	remaining: 48.3s
168:	learn: 0.0597670	total: 9.76s	remaining: 48s
169:	learn: 0.0597275	total: 9.78s	remaining: 47.7s
170:	learn: 0.0596460	total: 9.79s	remaining: 47.5s
171:	learn: 0.0

314:	learn: 0.0547342	total: 11.8s	remaining: 25.7s
315:	learn: 0.0547141	total: 11.8s	remaining: 25.6s
316:	learn: 0.0546952	total: 11.9s	remaining: 25.5s
317:	learn: 0.0546626	total: 11.9s	remaining: 25.5s
318:	learn: 0.0546515	total: 11.9s	remaining: 25.4s
319:	learn: 0.0546396	total: 11.9s	remaining: 25.3s
320:	learn: 0.0546321	total: 11.9s	remaining: 25.2s
321:	learn: 0.0546187	total: 11.9s	remaining: 25.1s
322:	learn: 0.0545895	total: 11.9s	remaining: 25s
323:	learn: 0.0545679	total: 11.9s	remaining: 24.9s
324:	learn: 0.0545501	total: 12s	remaining: 24.8s
325:	learn: 0.0545416	total: 12s	remaining: 24.7s
326:	learn: 0.0545238	total: 12s	remaining: 24.6s
327:	learn: 0.0545131	total: 12s	remaining: 24.6s
328:	learn: 0.0544997	total: 12s	remaining: 24.5s
329:	learn: 0.0544898	total: 12s	remaining: 24.4s
330:	learn: 0.0544685	total: 12s	remaining: 24.3s
331:	learn: 0.0544572	total: 12s	remaining: 24.2s
332:	learn: 0.0544493	total: 12.1s	remaining: 24.1s
333:	learn: 0.0544331	total: 1

477:	learn: 0.0528743	total: 14s	remaining: 15.3s
478:	learn: 0.0528677	total: 14s	remaining: 15.2s
479:	learn: 0.0528564	total: 14s	remaining: 15.2s
480:	learn: 0.0528486	total: 14s	remaining: 15.1s
481:	learn: 0.0528426	total: 14s	remaining: 15.1s
482:	learn: 0.0528359	total: 14.1s	remaining: 15s
483:	learn: 0.0528217	total: 14.1s	remaining: 15s
484:	learn: 0.0528142	total: 14.1s	remaining: 15s
485:	learn: 0.0528092	total: 14.1s	remaining: 14.9s
486:	learn: 0.0528034	total: 14.1s	remaining: 14.9s
487:	learn: 0.0527956	total: 14.1s	remaining: 14.8s
488:	learn: 0.0527918	total: 14.1s	remaining: 14.8s
489:	learn: 0.0527714	total: 14.1s	remaining: 14.7s
490:	learn: 0.0527652	total: 14.2s	remaining: 14.7s
491:	learn: 0.0527501	total: 14.2s	remaining: 14.6s
492:	learn: 0.0527433	total: 14.2s	remaining: 14.6s
493:	learn: 0.0527389	total: 14.2s	remaining: 14.5s
494:	learn: 0.0527338	total: 14.2s	remaining: 14.5s
495:	learn: 0.0527259	total: 14.2s	remaining: 14.5s
496:	learn: 0.0527054	total:

637:	learn: 0.0518974	total: 16.2s	remaining: 9.18s
638:	learn: 0.0518924	total: 16.2s	remaining: 9.15s
639:	learn: 0.0518868	total: 16.2s	remaining: 9.12s
640:	learn: 0.0518832	total: 16.2s	remaining: 9.1s
641:	learn: 0.0518791	total: 16.3s	remaining: 9.07s
642:	learn: 0.0518753	total: 16.3s	remaining: 9.04s
643:	learn: 0.0518715	total: 16.3s	remaining: 9.01s
644:	learn: 0.0518679	total: 16.3s	remaining: 8.98s
645:	learn: 0.0518617	total: 16.3s	remaining: 8.95s
646:	learn: 0.0518582	total: 16.4s	remaining: 8.93s
647:	learn: 0.0518551	total: 16.4s	remaining: 8.9s
648:	learn: 0.0518481	total: 16.4s	remaining: 8.88s
649:	learn: 0.0518406	total: 16.4s	remaining: 8.85s
650:	learn: 0.0518383	total: 16.4s	remaining: 8.82s
651:	learn: 0.0518364	total: 16.5s	remaining: 8.79s
652:	learn: 0.0518335	total: 16.5s	remaining: 8.76s
653:	learn: 0.0518274	total: 16.5s	remaining: 8.72s
654:	learn: 0.0518235	total: 16.5s	remaining: 8.69s
655:	learn: 0.0518188	total: 16.5s	remaining: 8.66s
656:	learn: 0.

...

244:	learn: 0.0567944	total: 3.45s	remaining: 10.6s
245:	learn: 0.0567662	total: 3.46s	remaining: 10.6s
246:	learn: 0.0567483	total: 3.47s	remaining: 10.6s
247:	learn: 0.0567115	total: 3.49s	remaining: 10.6s
248:	learn: 0.0566867	total: 3.5s	remaining: 10.6s
249:	learn: 0.0566788	total: 3.51s	remaining: 10.5s
250:	learn: 0.0566512	total: 3.52s	remaining: 10.5s
251:	learn: 0.0566167	total: 3.54s	remaining: 10.5s
252:	learn: 0.0566078	total: 3.55s	remaining: 10.5s
253:	learn: 0.0565757	total: 3.56s	remaining: 10.5s
254:	learn: 0.0565583	total: 3.58s	remaining: 10.4s
255:	learn: 0.0565312	total: 3.59s	remaining: 10.4s
256:	learn: 0.0565046	total: 3.61s	remaining: 10.4s
257:	learn: 0.0564733	total: 3.62s	remaining: 10.4s
258:	learn: 0.0564296	total: 3.64s	remaining: 10.4s
259:	learn: 0.0564122	total: 3.65s	remaining: 10.4s
260:	learn: 0.0563826	total: 3.66s	remaining: 10.4s
261:	learn: 0.0563662	total: 3.67s	remaining: 10.3s
262:	learn: 0.0563390	total: 3.69s	remaining: 10.3s
263:	learn: 0

406:	learn: 0.0537440	total: 12.4s	remaining: 18.1s
407:	learn: 0.0537312	total: 12.5s	remaining: 18.1s
408:	learn: 0.0537109	total: 12.5s	remaining: 18s
409:	learn: 0.0536929	total: 12.5s	remaining: 18s
410:	learn: 0.0536884	total: 12.5s	remaining: 17.9s
411:	learn: 0.0536830	total: 12.5s	remaining: 17.9s
412:	learn: 0.0536693	total: 12.6s	remaining: 17.9s
413:	learn: 0.0536594	total: 12.6s	remaining: 17.8s
414:	learn: 0.0536492	total: 12.6s	remaining: 17.8s
415:	learn: 0.0536409	total: 12.6s	remaining: 17.7s
416:	learn: 0.0536289	total: 12.6s	remaining: 17.7s
417:	learn: 0.0536245	total: 12.7s	remaining: 17.6s
418:	learn: 0.0536082	total: 12.7s	remaining: 17.6s
419:	learn: 0.0536020	total: 12.7s	remaining: 17.5s
420:	learn: 0.0535918	total: 12.7s	remaining: 17.5s
421:	learn: 0.0535674	total: 12.7s	remaining: 17.4s
422:	learn: 0.0535425	total: 12.7s	remaining: 17.4s
423:	learn: 0.0535375	total: 12.7s	remaining: 17.3s
424:	learn: 0.0535264	total: 12.8s	remaining: 17.3s
425:	learn: 0.05

572:	learn: 0.0522515	total: 14.8s	remaining: 11s
573:	learn: 0.0522402	total: 14.8s	remaining: 11s
574:	learn: 0.0522325	total: 14.9s	remaining: 11s
575:	learn: 0.0522241	total: 14.9s	remaining: 11s
576:	learn: 0.0522182	total: 14.9s	remaining: 10.9s
577:	learn: 0.0522159	total: 14.9s	remaining: 10.9s
578:	learn: 0.0522092	total: 14.9s	remaining: 10.9s
579:	learn: 0.0522017	total: 15s	remaining: 10.8s
580:	learn: 0.0521990	total: 15s	remaining: 10.8s
581:	learn: 0.0521963	total: 15s	remaining: 10.8s
582:	learn: 0.0521852	total: 15s	remaining: 10.7s
583:	learn: 0.0521772	total: 15s	remaining: 10.7s
584:	learn: 0.0521696	total: 15.1s	remaining: 10.7s
585:	learn: 0.0521641	total: 15.1s	remaining: 10.6s
586:	learn: 0.0521611	total: 15.1s	remaining: 10.6s
587:	learn: 0.0521550	total: 15.1s	remaining: 10.6s
588:	learn: 0.0521493	total: 15.1s	remaining: 10.6s
589:	learn: 0.0521424	total: 15.1s	remaining: 10.5s
590:	learn: 0.0521337	total: 15.2s	remaining: 10.5s
591:	learn: 0.0521294	total: 1

731:	learn: 0.0515073	total: 17s	remaining: 6.22s
732:	learn: 0.0515033	total: 17s	remaining: 6.19s
733:	learn: 0.0514989	total: 17s	remaining: 6.17s
734:	learn: 0.0514938	total: 17s	remaining: 6.14s
735:	learn: 0.0514896	total: 17s	remaining: 6.11s
736:	learn: 0.0514844	total: 17.1s	remaining: 6.09s
737:	learn: 0.0514827	total: 17.1s	remaining: 6.06s
738:	learn: 0.0514799	total: 17.1s	remaining: 6.03s
739:	learn: 0.0514753	total: 17.1s	remaining: 6.01s
740:	learn: 0.0514710	total: 17.1s	remaining: 5.98s
741:	learn: 0.0514673	total: 17.1s	remaining: 5.95s
742:	learn: 0.0514644	total: 17.1s	remaining: 5.93s
743:	learn: 0.0514612	total: 17.1s	remaining: 5.9s
744:	learn: 0.0514592	total: 17.2s	remaining: 5.88s
745:	learn: 0.0514562	total: 17.2s	remaining: 5.85s
746:	learn: 0.0514532	total: 17.2s	remaining: 5.83s
747:	learn: 0.0514511	total: 17.2s	remaining: 5.8s
748:	learn: 0.0514492	total: 17.2s	remaining: 5.78s
749:	learn: 0.0514428	total: 17.3s	remaining: 5.75s
750:	learn: 0.0514397	to

891:	learn: 0.0510122	total: 19.2s	remaining: 2.32s
892:	learn: 0.0510084	total: 19.2s	remaining: 2.3s
893:	learn: 0.0510058	total: 19.2s	remaining: 2.27s
894:	learn: 0.0510045	total: 19.2s	remaining: 2.25s
895:	learn: 0.0510021	total: 19.2s	remaining: 2.23s
896:	learn: 0.0509990	total: 19.2s	remaining: 2.21s
897:	learn: 0.0509972	total: 19.2s	remaining: 2.18s
898:	learn: 0.0509935	total: 19.2s	remaining: 2.16s
899:	learn: 0.0509905	total: 19.3s	remaining: 2.14s
900:	learn: 0.0509890	total: 19.3s	remaining: 2.12s
901:	learn: 0.0509853	total: 19.3s	remaining: 2.09s
902:	learn: 0.0509823	total: 19.3s	remaining: 2.07s
903:	learn: 0.0509790	total: 19.3s	remaining: 2.05s
904:	learn: 0.0509772	total: 19.3s	remaining: 2.03s
905:	learn: 0.0509752	total: 19.3s	remaining: 2.01s
906:	learn: 0.0509732	total: 19.3s	remaining: 1.98s
907:	learn: 0.0509713	total: 19.4s	remaining: 1.96s
908:	learn: 0.0509684	total: 19.4s	remaining: 1.94s
909:	learn: 0.0509658	total: 19.4s	remaining: 1.92s
910:	learn: 0

In [63]:
# Fit and evaluate the best model on the test set
best_model = CatBoostRegressor() # Replace with the best performing mod
best_model.fit(X_train, y_train)

test_preds = best_model.predict(df_test)
test_preds = np.exp(test_preds)-1

Learning rate set to 0.08835
0:	learn: 1.0392551	total: 17.6ms	remaining: 17.5s
1:	learn: 0.9515901	total: 32.8ms	remaining: 16.4s
2:	learn: 0.8716304	total: 47.5ms	remaining: 15.8s
3:	learn: 0.7988520	total: 62.2ms	remaining: 15.5s
4:	learn: 0.7332774	total: 77.1ms	remaining: 15.3s
5:	learn: 0.6731540	total: 92.2ms	remaining: 15.3s
6:	learn: 0.6187468	total: 109ms	remaining: 15.5s
7:	learn: 0.5696984	total: 124ms	remaining: 15.4s
8:	learn: 0.5258246	total: 139ms	remaining: 15.3s
9:	learn: 0.4856499	total: 153ms	remaining: 15.2s
10:	learn: 0.4490202	total: 168ms	remaining: 15.1s
11:	learn: 0.4165838	total: 188ms	remaining: 15.5s
12:	learn: 0.3874911	total: 205ms	remaining: 15.6s
13:	learn: 0.3614922	total: 221ms	remaining: 15.6s
14:	learn: 0.3377955	total: 236ms	remaining: 15.5s
15:	learn: 0.3163983	total: 251ms	remaining: 15.4s
16:	learn: 0.2968334	total: 266ms	remaining: 15.4s
17:	learn: 0.2796875	total: 280ms	remaining: 15.3s
18:	learn: 0.2642561	total: 296ms	remaining: 15.3s
19:	le

164:	learn: 0.0609165	total: 2.76s	remaining: 14s
165:	learn: 0.0608370	total: 2.77s	remaining: 13.9s
166:	learn: 0.0607940	total: 2.79s	remaining: 13.9s
167:	learn: 0.0606857	total: 2.8s	remaining: 13.9s
168:	learn: 0.0606172	total: 2.82s	remaining: 13.9s
169:	learn: 0.0605446	total: 2.83s	remaining: 13.8s
170:	learn: 0.0604898	total: 2.85s	remaining: 13.8s
171:	learn: 0.0604158	total: 2.86s	remaining: 13.8s
172:	learn: 0.0603261	total: 2.87s	remaining: 13.7s
173:	learn: 0.0602557	total: 2.89s	remaining: 13.7s
174:	learn: 0.0602209	total: 2.9s	remaining: 13.7s
175:	learn: 0.0601722	total: 2.91s	remaining: 13.6s
176:	learn: 0.0600917	total: 2.93s	remaining: 13.6s
177:	learn: 0.0599542	total: 2.95s	remaining: 13.6s
178:	learn: 0.0598631	total: 2.96s	remaining: 13.6s
179:	learn: 0.0597541	total: 2.98s	remaining: 13.6s
180:	learn: 0.0596973	total: 2.99s	remaining: 13.5s
181:	learn: 0.0596277	total: 3.01s	remaining: 13.5s
182:	learn: 0.0595625	total: 3.02s	remaining: 13.5s
183:	learn: 0.05

326:	learn: 0.0551236	total: 5.14s	remaining: 10.6s
327:	learn: 0.0551102	total: 5.15s	remaining: 10.6s
328:	learn: 0.0550990	total: 5.17s	remaining: 10.5s
329:	learn: 0.0550751	total: 5.19s	remaining: 10.5s
330:	learn: 0.0550306	total: 5.21s	remaining: 10.5s
331:	learn: 0.0550166	total: 5.22s	remaining: 10.5s
332:	learn: 0.0550054	total: 5.24s	remaining: 10.5s
333:	learn: 0.0549884	total: 5.25s	remaining: 10.5s
334:	learn: 0.0549708	total: 5.26s	remaining: 10.4s
335:	learn: 0.0549610	total: 5.28s	remaining: 10.4s
336:	learn: 0.0549482	total: 5.29s	remaining: 10.4s
337:	learn: 0.0549395	total: 5.31s	remaining: 10.4s
338:	learn: 0.0549256	total: 5.32s	remaining: 10.4s
339:	learn: 0.0549083	total: 5.34s	remaining: 10.4s
340:	learn: 0.0548947	total: 5.35s	remaining: 10.3s
341:	learn: 0.0548871	total: 5.37s	remaining: 10.3s
342:	learn: 0.0548738	total: 5.38s	remaining: 10.3s
343:	learn: 0.0548684	total: 5.4s	remaining: 10.3s
344:	learn: 0.0548569	total: 5.41s	remaining: 10.3s
345:	learn: 0

495:	learn: 0.0531542	total: 8.08s	remaining: 8.21s
496:	learn: 0.0531408	total: 8.1s	remaining: 8.2s
497:	learn: 0.0531350	total: 8.11s	remaining: 8.18s
498:	learn: 0.0531204	total: 8.13s	remaining: 8.16s
499:	learn: 0.0531132	total: 8.14s	remaining: 8.14s
500:	learn: 0.0531103	total: 8.15s	remaining: 8.12s
501:	learn: 0.0531025	total: 8.17s	remaining: 8.1s
502:	learn: 0.0530933	total: 8.18s	remaining: 8.09s
503:	learn: 0.0530878	total: 8.2s	remaining: 8.07s
504:	learn: 0.0530822	total: 8.21s	remaining: 8.05s
505:	learn: 0.0530792	total: 8.22s	remaining: 8.03s
506:	learn: 0.0530702	total: 8.24s	remaining: 8.01s
507:	learn: 0.0530579	total: 8.26s	remaining: 8s
508:	learn: 0.0530517	total: 8.27s	remaining: 7.98s
509:	learn: 0.0530414	total: 8.29s	remaining: 7.96s
510:	learn: 0.0530368	total: 8.3s	remaining: 7.94s
511:	learn: 0.0530353	total: 8.31s	remaining: 7.92s
512:	learn: 0.0530309	total: 8.33s	remaining: 7.91s
513:	learn: 0.0530283	total: 8.34s	remaining: 7.89s
514:	learn: 0.053023

655:	learn: 0.0521954	total: 17.7s	remaining: 9.27s
656:	learn: 0.0521916	total: 17.7s	remaining: 9.24s
657:	learn: 0.0521826	total: 17.7s	remaining: 9.21s
658:	learn: 0.0521774	total: 17.7s	remaining: 9.17s
659:	learn: 0.0521738	total: 17.7s	remaining: 9.14s
660:	learn: 0.0521707	total: 17.8s	remaining: 9.1s
661:	learn: 0.0521635	total: 17.8s	remaining: 9.07s
662:	learn: 0.0521542	total: 17.8s	remaining: 9.04s
663:	learn: 0.0521517	total: 17.8s	remaining: 9.01s
664:	learn: 0.0521491	total: 17.8s	remaining: 8.98s
665:	learn: 0.0521471	total: 17.8s	remaining: 8.95s
666:	learn: 0.0521436	total: 17.9s	remaining: 8.92s
667:	learn: 0.0521408	total: 17.9s	remaining: 8.89s
668:	learn: 0.0521379	total: 17.9s	remaining: 8.86s
669:	learn: 0.0521310	total: 17.9s	remaining: 8.83s
670:	learn: 0.0521287	total: 17.9s	remaining: 8.8s
671:	learn: 0.0521192	total: 18s	remaining: 8.77s
672:	learn: 0.0521149	total: 18s	remaining: 8.73s
673:	learn: 0.0521103	total: 18s	remaining: 8.7s
674:	learn: 0.0521065

819:	learn: 0.0515747	total: 20.7s	remaining: 4.55s
820:	learn: 0.0515729	total: 20.7s	remaining: 4.52s
821:	learn: 0.0515691	total: 20.8s	remaining: 4.5s
822:	learn: 0.0515664	total: 20.8s	remaining: 4.47s
823:	learn: 0.0515641	total: 20.8s	remaining: 4.44s
824:	learn: 0.0515623	total: 20.8s	remaining: 4.41s
825:	learn: 0.0515594	total: 20.8s	remaining: 4.38s
826:	learn: 0.0515573	total: 20.8s	remaining: 4.36s
827:	learn: 0.0515552	total: 20.8s	remaining: 4.33s
828:	learn: 0.0515531	total: 20.9s	remaining: 4.3s
829:	learn: 0.0515494	total: 20.9s	remaining: 4.28s
830:	learn: 0.0515466	total: 20.9s	remaining: 4.25s
831:	learn: 0.0515438	total: 20.9s	remaining: 4.22s
832:	learn: 0.0515385	total: 20.9s	remaining: 4.19s
833:	learn: 0.0515356	total: 20.9s	remaining: 4.17s
834:	learn: 0.0515318	total: 21s	remaining: 4.14s
835:	learn: 0.0515295	total: 21s	remaining: 4.11s
836:	learn: 0.0515260	total: 21s	remaining: 4.08s
837:	learn: 0.0515236	total: 21s	remaining: 4.06s
838:	learn: 0.0515223	

985:	learn: 0.0511453	total: 23.6s	remaining: 334ms
986:	learn: 0.0511423	total: 23.6s	remaining: 311ms
987:	learn: 0.0511407	total: 23.6s	remaining: 287ms
988:	learn: 0.0511390	total: 23.6s	remaining: 263ms
989:	learn: 0.0511365	total: 23.6s	remaining: 239ms
990:	learn: 0.0511337	total: 23.6s	remaining: 215ms
991:	learn: 0.0511314	total: 23.6s	remaining: 191ms
992:	learn: 0.0511292	total: 23.7s	remaining: 167ms
993:	learn: 0.0511275	total: 23.7s	remaining: 143ms
994:	learn: 0.0511262	total: 23.7s	remaining: 119ms
995:	learn: 0.0511239	total: 23.7s	remaining: 95.2ms
996:	learn: 0.0511222	total: 23.7s	remaining: 71.4ms
997:	learn: 0.0511185	total: 23.7s	remaining: 47.6ms
998:	learn: 0.0511161	total: 23.8s	remaining: 23.8ms
999:	learn: 0.0511145	total: 23.8s	remaining: 0us


In [64]:
# Generate predictions for the test data using the previously fit best mi
df_test = pd.read_csv('test.csv')
id1 = df_test['id' ]
df_test['pred']=test_preds
preds = df_test['pred']

# Create a submission DataFrame with the "datetime column and predicted
submission = pd. DataFrame({'id': id1, 'num_sold': preds})

submission

,id,num_sold
0,136950,41.812575
1,136951,40.515558
2,136952,6.346333
3,136953,38.858723
4,136954,31.715093
...,...,...
27370,164320,722.237009
27371,164321,735.466059
27372,164322,113.351831
27373,164323,674.377308


In [65]:
# Save the submission DataFrame as a CS file without +including the index)
submission.to_csv( 'submission.csv', index=False)